In [1]:
!python -c 'import keras; print(keras.__version__)'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'keras'


In [2]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
import numpy as np

num_samples = 1000
height = 224
width = 224
num_classes = 1000

G=2

# Instantiate the base model (or "template" model).
# We recommend doing this with under a CPU device scope,
# so that the model's weights are hosted on CPU memory.
# Otherwise they may end up hosted on a GPU, which would
# complicate weight sharing.
with tf.device('/cpu:0'):
    model = Xception(weights=None,
                     input_shape=(height, width, 3),
                     classes=num_classes)

# Replicates the model on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=G)
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer='rmsprop')

# Generate dummy data.
x = np.random.random((num_samples, height, width, 3))
y = np.random.random((num_samples, num_classes))

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=10, batch_size=32*G)

# Save model via the template model (which shares the same weights):
#model.save('my_model.h5')



ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
import numpy as np

num_samples = 1000
height = 224
width = 224
num_classes = 1000

G=1

model = Xception(weights=None,
                     input_shape=(height, width, 3),
                     classes=num_classes)

model.compile(loss='categorical_crossentropy',
                       optimizer='rmsprop')

# Generate dummy data.
x = np.random.random((num_samples, height, width, 3))
y = np.random.random((num_samples, num_classes))

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
model.fit(x, y, epochs=10, batch_size=32*G)

# Save model via the template model (which shares the same weights):
#model.save('my_model.h5')




Epoch 1/10
1000/1000 [==============================] - 19s 19ms/step - loss: 3462.4697
Epoch 2/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3457.4632
Epoch 3/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3457.3616
Epoch 4/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3457.2924
Epoch 5/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3457.0518
Epoch 6/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3456.2236
Epoch 7/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3454.8189
Epoch 8/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3453.3024
Epoch 9/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3451.0811
Epoch 10/10
1000/1000 [==============================] - 15s 15ms/step - loss: 3448.6749


In [3]:
import tensorflow as tf
from keras.utils import multi_gpu_model
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

G=2

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(4000, input_dim=8000, activation='tanh'))
    model.add(Dense(2000, input_dim=8000, activation='relu'))
    for i in range(0,10):
        model.add(Dense(500, activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
print (model.summary())
model =  multi_gpu_model(model, G)
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
x = np.random.rand(131072, 8000)
y = np.random.randint(0, 2, (131072, 1))
model.fit(x, y, epochs=5, batch_size=2048*G)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4000)              32004000  
_________________________________________________________________
dense_7 (Dense)              (None, 2000)              8002000   
_________________________________________________________________
dense_8 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_9 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_10 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_11 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_12 (Dense)             (None, 500)               250500    
__________

In [4]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

G=1

model = Sequential()
model.add(Dense(4000, input_dim=8000, activation='tanh'))
model.add(Dense(2000, input_dim=8000, activation='relu'))
for i in range(0,10):
    model.add(Dense(500, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print (model.summary())
#model =  multi_gpu_model(model, G)
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
x = np.random.rand(131072, 8000)
y = np.random.randint(0, 2, (131072, 1))
model.fit(x, y, epochs=5, batch_size=2048*G)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 4000)              32004000  
_________________________________________________________________
dense_21 (Dense)             (None, 2000)              8002000   
_________________________________________________________________
dense_22 (Dense)             (None, 500)               1000500   
_________________________________________________________________
dense_23 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_24 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_25 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_26 (Dense)             (None, 500)               250500    
__________

In [ ]:
G=2
import tensorflow as tf
print("[INFO] training with {} GPUs...".format(G))
from keras.utils.training_utils import multi_gpu_model
 
# we'll store a copy of the model on *every* GPU and then combine
# the results from the gradient updates on the CPU
with tf.device("/cpu:0"):
    # initialize the model
    model = Sequential()
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    # make the model parallel
    model = multi_gpu_model(model, gpus=G)
    # initialize the optimizer and model
    print("[INFO] compiling model...")
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    # train the network
    print("[INFO] training network...")
    model.fit(x_train, y_train,
              batch_size=batch_size * G,
              epochs=epochs,
              validation_data=(x_test, y_test))